In [1]:
print('help world!!')

help world!!


In [1]:
import transformers

In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
import transformers

In [8]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [9]:

data = load_pdf("pdfs/")

In [5]:
import os

os.getcwd()

'c:\\Users\\mayuo\\OneDrive\\Documents\\Machine Learning by Abraham\\chatbot\\research'

In [6]:
%cd ..

c:\Users\mayuo\OneDrive\Documents\Machine Learning by Abraham\chatbot


c:\Users\mayuo\OneDrive\Documents\Machine Learning by Abraham\chatbot\env\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [7]:
os.getcwd()

'c:\\Users\\mayuo\\OneDrive\\Documents\\Machine Learning by Abraham\\chatbot'

In [50]:
data[3]

Document(metadata={'source': 'pdfs\\fitness.pdf', 'page': 3}, page_content='v3.1\n')

In [10]:
## create chuncks 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,
                                                chunk_overlap=20
)

In [11]:
text_chunks = text_splitter.split_documents(data) 

In [12]:
len(text_chunks)

954

In [13]:
# downloading the embedding
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [14]:
embeddings = download_hugging_face_embeddings()
embeddings

c:\Users\mayuo\OneDrive\Documents\Machine Learning by Abraham\chatbot\env\lib\site-packages\langchain_core\_api\deprecation.py:151: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\mayuo\OneDrive\Documents\Machine Learning by Abraham\chatbot\env\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [15]:
query_result = embeddings.embed_query("this is a chat model")
print("Length", len(query_result))

Length 384


In [17]:
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [18]:
from pinecone import Pinecone as pc

In [19]:
pine = pc(api_key= PINECONE_API_KEY)

In [20]:
index_name = 'fitnesschatbot'

In [21]:
docsearch = Pinecone.from_texts(
    texts=[t.page_content for t in text_chunks],
    embedding=embeddings,
    index_name=index_name
    #pinecone_api_key=PINECONE_API_KEY
)

In [22]:
docsearch.as_retriever()

VectorStoreRetriever(tags=['Pinecone', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.pinecone.Pinecone object at 0x000001A9165DEB60>)

In [23]:
query = 'What are the top 3 ways of growing my chest'

In [24]:
doc = docsearch.similarity_search(query)
doc

[Document(page_content='THE BEST FULL-GYM\nCHEST WORKOUT \n[\noption\nB\n]\nWORKOUT #40\n    \nBY HANY RAMBOD\nIf you’ve been working out for a while, this routine will probably\nremind you of some of the chest sessions you did in the early days.\nIt’s good, old-fashioned hard work and provides a lot of isolation of the pecs.\nIf you’ve gotten more scientific with your training since and noticed fewer\ngains, this simple and direct approach may be what you need to get growing\nagain.\nHOW IT WORKS'),
 Document(page_content='HOW IT WORKS\n The more muscle you add to one side of your body, the\nmore you need to add to the opposite side in order to keep the balance. If\nyou don’t, the body will eventually forbid muscle growth to the area that has\ngrown out of whack, because it doesn’t want to risk the injury. This is one of\nthe most common and easily preventable ways to plateau.\nYou’ll target your chest very specifically in this routine\nwith moves like the three-way flye, which is bor

In [25]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [26]:
PROMPT = PromptTemplate(template = prompt_template,
                       input_variables = ['context','question'])
chain_type_kwargs={"prompt": PROMPT}

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "C:/Users/mayuo/.llama/checkpoints/Meta-Llama3.1-8B"

# Download tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


: 

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "C:/Users/mayuo/.llama/checkpoints/Meta-Llama3.1-8B"

# Download tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name,)



In [1]:
from transformers import AutoModelForCausalLM
model_name = "C:/Users/mayuo/.llama/checkpoints/Meta-Llama3.1-8B"
model = AutoModelForCausalLM.from_pretrained(model_name)

: 